In [68]:
import open3d as o3d
import numpy as np
import random
from sklearn.neighbors import NearestNeighbors
import pandas as pd
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
import os


In [69]:
# Import the coordinates and the features that have been generated by dMaSIF-site of the following proteins
#   - Subunit A (IgG Fc)
#   - Subunit C (GB1 protein)
#   - Complex (Subunit A and C combined)

%cd -q masif_site_outputs
coords_A = np.load("1fcc_A_predcoords.npy")
features_A = np.load("1fcc_A_predfeatures_emb1.npy")

coords_C = np.load("1fcc_C_predcoords.npy")
features_C = np.load("1fcc_C_predfeatures_emb1.npy")

coords_complex = np.load("1fcc_A, C_predcoords.npy")
features_complex = np.load("1fcc_A, C_predfeatures_emb1.npy")

[WinError 2] The system cannot find the file specified: 'masif_site_outputs'


In [70]:
coords_A.shape, coords_C.shape, coords_complex.shape

((25600, 3), (7317, 3), (31156, 3))

In [71]:
#create a PointCloud and add the points

pointcloud_A = o3d.geometry.PointCloud()
pointcloud_A.points = o3d.utility.Vector3dVector(coords_A)
o3d.visualization.draw_geometries([pointcloud_A])

pointcloud_C = o3d.geometry.PointCloud()
pointcloud_C.points = o3d.utility.Vector3dVector(coords_C)
o3d.visualization.draw_geometries([pointcloud_C])

pointcloud_complex = o3d.geometry.PointCloud()
pointcloud_complex.points = o3d.utility.Vector3dVector(coords_complex)
o3d.visualization.draw_geometries([pointcloud_complex])

In [72]:
# Create color vector for pointclouds
blue = [0, 0.5, 1]
red = [1, 0, 0]
green = [0, 1, 0]
colors_A = np.zeros((len(coords_A),3))
colors_C = np.zeros((len(coords_C),3))
colors_complex = np.zeros((len(coords_complex),3))
colors_A[:] = blue
colors_C[:] = blue
colors_complex = blue

In [73]:
def extract_buried_points(subunit_coords, complex_coords, threshold):

    '''Function that takes as input the coordinates of two similar pointclouds (one subunit and
    one complex involving that subunit) and a threshold. Returns a list of indeces indicating 
    which points of the first pointcloud (subunit) are more than a certain distance (threshold)
    away from any point in the second pointcloud. These points are "buried" in the complex. The 
    point with the largest distance above the threshold is returned as the "center" of the buried patch'''

    neigh = NearestNeighbors(n_neighbors=1)
    neigh.fit(complex_coords) 

    buried= []
    largest_dist = 0
    for i in range(len(subunit_coords)):
        if (dist := neigh.kneighbors([subunit_coords[i]])[0][0][0]) > threshold:
        #if neigh.kneighbors([subunit_coords[i]])[0][0][0] > threshold:
            buried.append(i)
            if dist > largest_dist:
                largest_dist = dist
                center = i
    return buried, center

In [74]:
buried_A, center_A = extract_buried_points(coords_A, coords_complex, 2)
buried_C, center_C = extract_buried_points(coords_C, coords_complex, 2)


In [75]:
print(center_A, center_C)

4419 6820


In [76]:
# Color pointcloud points according to whether they are buried (red) or not (blue)

colors_A[buried_A]=red
colors_C[buried_C]=red

colors_A[center_A] = green
colors_C[center_C] = green

pointcloud_A.colors = o3d.utility.Vector3dVector(colors_A)
pointcloud_C.colors = o3d.utility.Vector3dVector(colors_C)

In [77]:
o3d.visualization.draw_geometries([pointcloud_A])
o3d.visualization.draw_geometries([pointcloud_C])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [78]:
np.save("buried_A_coords.npy", buried_A)
np.save("buried_C_coords.npy", buried_C)